## This notebook shows creating a basic model for classification, and then using that model on data for inferencing

In [4]:
import azureml.core
import pandas as pd
from azureml.core import Workspace, Dataset
from azureml.core import Model
import joblib

# Load the workspace from the saved config file
ws = Workspace.from_config()
print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))

Ready to use Azure ML 1.36.0 to work with mm-aml


In [5]:
# Import libraries
from azureml.core import Experiment
from azureml.core import Model
from azureml.core import Run
import pandas as pd
import numpy as np
import joblib
import os
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.naive_bayes import GaussianNB
from sklearn.pipeline import Pipeline

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline

# Create an Azure ML experiment in your workspace
experiment = Experiment(workspace=ws, name= 'tester2')
run = experiment.start_logging()
print("Starting experiment:", experiment.name)

def summarize_classification(y_test, y_pred, run):
    acc = accuracy_score(y_test, y_pred, normalize=True) #how many predictions correct %
    num_acc = accuracy_score(y_test, y_pred, normalize = False)
    prec = precision_score(y_test, y_pred, average = 'weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    
    run.log('acc count', num_acc)
    run.log('Accuracy', acc)
    run.log('prec', prec)
    run.log('recall', recall)
    
    print('aacuracy count:', num_acc)
    print('accuracy score:', acc)
    print('precision:', prec)
    print('recall:', recall)
    

def model_train(ds_df, run):
    
    X = ds_df['text']
    Y = ds_df['labels']
    print('type of X')
    print(type(X))
    #sklearn pipeline
    clf = Pipeline([
                            ('count_vectorizer', CountVectorizer()),
                            ('classifier', LogisticRegression(solver='lbfgs', max_iter=10000))
                        ])
    #output of convectorizer, feed to classifier
    x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2)
    print('type of x_test')
    print(type(x_test))
    model = clf.fit(x_train, y_train)
    y_pred = model.predict(x_test)
    
    print('*************************')
    print('model predictions:')
    print(y_pred)
    summarize_classification(y_test, y_pred, run)

    return model


# load the diabetes dataset
print("Loading Data...")
dataset = Dataset.get_by_name(ws, name='email_dataset')
data = dataset.to_pandas_dataframe()
model = model_train(data, run)



# Save the trained model in the outputs folder
os.makedirs('outputs', exist_ok=True)
# note file saved in the outputs folder is automatically uploaded into experiment record

model_file = 'outputs/model.pkl'
joblib.dump(value=model, filename=model_file)

run.complete()



# Register the model
run.register_model(model_path=model_file, model_name= 'tester',
                   tags={'Model Type':'Logistic Regresssion'})

print('Model trained and registered.')

Starting experiment: tester2
Loading Data...
type of X
<class 'pandas.core.series.Series'>
type of x_test
<class 'pandas.core.series.Series'>
*************************
model predictions:
['spam' 'ham' 'ham' 'ham' 'ham' 'ham' 'spam' 'ham' 'ham' 'ham' 'ham' 'ham'
 'ham' 'ham' 'ham' 'ham' 'spam' 'ham' 'ham' 'ham' 'spam' 'ham' 'ham' 'ham'
 'ham' 'ham' 'ham' 'ham' 'ham' 'ham' 'ham' 'ham' 'ham' 'ham' 'ham' 'ham'
 'ham' 'ham' 'ham' 'ham' 'spam' 'ham' 'ham' 'ham' 'spam' 'ham' 'ham' 'ham'
 'spam' 'ham' 'ham' 'ham' 'ham' 'spam' 'ham' 'ham' 'spam' 'ham' 'ham'
 'ham' 'ham' 'spam' 'ham' 'ham' 'ham' 'ham' 'ham' 'ham' 'spam' 'ham' 'ham'
 'ham' 'ham' 'ham' 'ham' 'ham' 'ham' 'ham' 'ham' 'ham' 'ham' 'ham' 'ham'
 'ham' 'spam' 'ham' 'ham' 'ham' 'ham' 'ham' 'spam' 'ham' 'ham' 'ham' 'ham'
 'ham' 'ham' 'ham' 'ham' 'ham' 'spam' 'ham' 'ham' 'ham' 'ham' 'ham' 'ham'
 'ham' 'ham' 'ham' 'ham' 'ham' 'ham' 'ham' 'spam' 'spam' 'ham' 'ham' 'ham'
 'ham' 'ham' 'ham' 'ham' 'ham' 'ham' 'ham' 'ham' 'ham' 'ham' 'ham' 'ham'


In [6]:
# Get model from workspace - the code below will always retrieve the latest version of the model; specific versions can be targeted.
model_name = 'tester'
model_list = Model.list(ws, name=model_name, latest=True)
model_path = model_list[0].download(exist_ok=True)
model = joblib.load(model_path)
#results = model.predict(X)
#print(len(results))

In [7]:
result = model.predict(['Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...'])

In [8]:
result

array(['ham'], dtype=object)

In [9]:
data = pd.read_csv('./datasets/3000.csv', header=None)

In [16]:
print(data)

                                                   0
0  This message is free. Welcome to the new & imp...


In [10]:
data[0][0]

'This message is free. Welcome to the new & improved Sex & Dogging club! To unsubscribe from this service reply STOP. msgs@150p 18+only'

In [11]:
data[0]

0    This message is free. Welcome to the new & imp...
Name: 0, dtype: object

In [15]:
result = model.predict(data[0])

In [13]:
result[0]

'spam'